# Chapter 1: Sockets

## socket для протокола UDP

- В отличии от протокола TCP, в UDP мы не устанавливаем соединение между клиентом и сервером. Поэтому, клиент отправляет данные на какой-то сервер и не пытается убедиться получит он их или нет. Сервер же слушает свой порт и тоже не переживает получит он на него данные или нет.
- Также сервер не создает очереди подключения к порту. Серверу все равно кто, что и когда ему отправил на порт.

In [ ]:
# пример серверного приложения с протоколом UDP
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(('127.0.0.1', 8888))
result = sock.recv(1024) # ждет сообщение
print('Message: ', result.decode('utf-8'))
sock.close()

1. `sock = socket.socket(..)` - cоздается объект-порт.
    - `socket.AF_INET` - используется протокол IPv4, `socket.AF_INET6` - протокол IPv6
    - `socket.SOCK_DGRAM` - протокол UDP (User Datagram Protocol). DGRAM - т.е. DataGram
2. `sock.bind(('127.0.0.1', 8888))` - порт привязывается к серверу (IP = *127.0.0.1*; это локальный IP) и порту (*8888*) на нем, если он не занят системой. Если занят будет выдана ошибка. Если вместо IP укзать `''` (пустую строку), либо `0`, либо `0.0.0.0`, то резервируются все локальные хосты. 
3. `result = sock.recv(1024)` - recv = receive. Принимает пакет указанного размера (в нашем случаии *1024*). На этой команде программа "слушает" порт т.е. ждет сообщение и не проходит далее.

In [ ]:
# пример клиентского приложения с протоколом UDP
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.sendto(b'Grond control to Major Tom', ('127.0.0.1', 8888))
sock.close()

- `sock.sendto(b'Grond control to Major Tom', ('127.0.0.1', 8888))` - отправляет **байтовый объект** (поэтому строка представлена в байтовом виде `b'aaa'`) на сокет *8888* по IP = *'127.0.0.1'*. Если вместо IP укзать `'localhost'`, то программа будет искать порт на локальной машине.

## unix sockets

**unix sockets** - это сокет, который использует не сетевой порт для обмена данными между клиентом и сервером, а файл. Т.е. клиент пишет что-то в файл, а сервер считывает этот файл.     

- Поддерживаются на UNIX системах.
- Обещают поддрежку на Windows в будущем.

Отличие от обычных сокетов в том, что в конструкторе сокета надо использовать `socket.AF_UNIX`. А также, в методе `.bind(..)` пишется не IP:port, а путь до файла.    

In [ ]:
# пример серверного приложения с unix sockets
import socket
import os

unix_sock_name = 'unix.sock'
sock = socket.socket(socket.AF_UNIX, socket.SOCK_DGRAM)

if os.path.exists(unix_sock_name):
    os.remove(unix_sock_name)

sock.bind(unix_sock_name)

while True:
    try:
        result = sock.recv(1024) # ждет сообщение
    except KeyboardInterrupt:
        sock.close()
        break
    else:
        print('Message: ', result.decode('utf-8'))

Тут сервер считывает данные сокета в бесконечном цикле, пока не будут нажаты Control+C или Delete. 

In [ ]:
# пример клиентского приложения с unix sockets
import socket

unix_sock_name = 'unix.sock'
sock = socket.socket(socket.AF_UNIX, socket.SOCK_DGRAM)
sock.sendto(b'Grond control to Major Tom', unix_sock_name)
sock.close()

## socket для протокола TCP

- Протокол в начале создает соединение между клиентом и сервером. Поэтому, в питоне мы один раз создаем соединение и больше не указыва
- Протокол создает очереди клиентов для общения с сервером. Первый клиент отправляет данные серверу, остальные клиенты ждут своей очереди. Сервер общается только с одним клиентом за раз. 

In [ ]:
# пример серверного приложения с протоколом TCP
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('127.0.0.1', 55555))
sock.listen(5)

while True:
    try:
        client, address = sock.accept()
    except KeyboardInterrupt:
        sock.close()
        break
    else:
        result = client.recv(1024)
        client.close()
        print('Message:', result.decode('utf-8'))        

1. В конструкторе мы ставим `socket.SOCK_STREAM` - это протокол TCP. 
2. `sock.bind(('127.0.0.1', 55555))` - далее привязываем сокет к IP и порту.
3. `sock.listen(5)` - указываем, что в очереди на отправку данных может быть не более *5* клиентов. 
4. `client, address = sock.accept()` - обрабатываем клиента из очереди. 
    - `client` - это сокет. Надо различать: `sock` - это сокет сервера. `client` - это клиентский сокет. При этом, ОС выделяет отдельный локальный порт для каждого клиента (с помощью команды `accept()`). Далее, вся работа с клиентским сокетом идет стандартно.
    - `address` - это кортеж с IP адресом клиент и выделенным ему локальным портом.
5. `result = client.recv(1024)` - принимаем от клиентского сокета данные как от обычного сокета. 
6. `client.close()` - после обработки клиента, его надо обязательно закрыть.

In [ ]:
# пример клиентского приложения с протоколом TCP
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('127.0.0.1', 55555))
sock.send(b'Grond control to Major Tom')
sock.close()

1. Создаем клиентский сокет.
2. `sock.connect(('127.0.0.1', 55555))` - создаем соединение с серверным сокетом по соответствующему адресу.
3. `sock.send(b'Grond control to Major Tom')` - т.к. соединение уже установлено, то клиент может отправлять сообщения, не указывая адрес сервера еще раз.